Import libraries

In [ ]:
import pandas as pd
import numpy as np
import time
from itertools import groupby
from itertools import combinations
from itertools import permutations

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df = pd.read_csv('steam-200k.csv',header=None,converters={i: str for i in range(1000)})

In [ ]:
df = df[:1000]
selected_df =  df.iloc[:,0:2]

In [ ]:
removed_dup_df = selected_df.drop_duplicates(subset=[0,1])

In [ ]:
transaction = pd.crosstab(index = removed_dup_df[0], columns= removed_dup_df[1])

In [ ]:
transaction.head(10)

1,140,404Sight,7 Days to Die,A Story About My Uncle,A Virus Named TOM,ARK Survival Evolved,AaaaaAAaaaAAAaaAAAAaAAAAA!!! for the Awesome,Afterlife Empire,Age of Empires II HD Edition,Age of Empires II HD The Forgotten,Age of Empires III Complete Collection,Age of Mythology Extended Edition,Air Brawl,Alan Wake,Alice Madness Returns,Alien Isolation,Alien Swarm,America's Army 3,America's Army Proving Grounds,Amnesia A Machine for Pigs,Amnesia The Dark Descent,Anachronox,And Yet It Moves,Antichamber,App Game Kit,ArcheAge,Arma 2,Arma 2 British Armed Forces,Arma 2 DayZ Mod,Arma 2 Operation Arrowhead,Arma 2 Operation Arrowhead Beta (Obsolete),Arma 2 Private Military Company,Assassin's Creed II,Assassin's Creed III,Assassin's Creed IV Black Flag,Astebreed,Audiosurf,Aveyond 3-1 Lord of Twilight,Awesomenauts,Axis Game Factory's AGFPRO - Drone Kombat FPS Multi-Player DLC,...,Titan Quest Immortal Throne,Tom Clancy's Splinter Cell Chaos Theory,Tomb Raider,Torchlight,Toribash,Total War SHOGUN 2,Transistor,Transistor Soundtrack,Trine,Tropico 4,Turbo Dismount,TypeRider,Ultra Street Fighter IV,Undertale,Unepic,Unholy Heights,Unturned,VVVVVV,Valkyria Chronicles,Valkyria Chronicles Challenge of the Edy Detachment,Valkyria Chronicles Edy's Mission Enter the Edy Detachment,Valkyria Chronicles Hard EX Mode,Valkyria Chronicles Selveria's Mission Behind Her Blue Flame,Vessel,Viscera Cleanup Detail Shadow Warrior,WRC 4 FIA WORLD RALLY CHAMPIONSHIP,War Thunder,Warframe,"Warhammer 40,000 Dawn of War - Game of the Year Edition",Watch_Dogs,Whisper of a Rose,Wizorb,Wolfenstein The New Order,Wrack,Ys Origin,Zen Bound 2,Ziggurat,Zombies Monsters Robots,resident evil 4 / biohazard 4,theHunter
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
126340495,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0
140954425,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
144138643,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
150128162,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
151603712,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
159538705,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
167362888,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
170625356,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
176410694,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Create frequent itemset

In [ ]:
def create_Frequent_Itemset(data):

#define maximum lenght of a set and minimum support
  max_length = 3
  min_sup = 0.06

#create empty dict to store element: (items : support)
  support = {} 

#transform data to a list
  L = list(data.columns)

#iteration
#generating combination of items with lenght i in ith iteration
  for i in range(1, max_length+1):

    #create itemsets by combination
    c = set(combinations(L,i))

    #reset L for next iteration 
    L = set()
    
    #iterate through every item in c
    for j in list(c):

      #calculate support. product == and in horizontal. sum to calculate frequency in vertical
      sup = data.loc[:,j].product(axis=1).sum()/len(data.index)

      if(sup > min_sup): #compare with min_sup
        support[j] = sup

        #union
        L = set(set(L) | set(j))
        
  #transfomr into dataframe
  result = pd.DataFrame(list(support.items()), columns = ["Items", "Support"])
  
  return(result)

In [ ]:
#my_freq_itemset = create_Frequent_Itemset(transaction)

In [ ]:
#my_freq_itemset.head()

In [ ]:
def create_Assocition_Rules(data):
  min_threshold = 0.5

#create dict (item : support)
  support = pd.Series(data.Support.values, index=data.Items).to_dict() 

  #create output list
  final_data = []

  #get Items columns in transaction
  L = data.Items.values

  #generating rule using permutation
  item_permutations = list(permutations(L,2))

  #Iterating each rule
  for per in item_permutations:

    #If Antecedent of rule is subset of super set(Atecedent, Consequent)
    if set(per[0]).issubset(per[1]): 

      #computing confidence
      conf = support[per[1]]/support[per[0]]
      if conf > min_threshold:  

        #get the rest part of an itemset - Consequent
        j = per[1][not per[1].index(per[0][0])]
        final_data.append([per, per[0], (j,), support[per[0]], support[(j,)], support[per[1]], conf])
  
  #transform into a dataframe
  result = pd.DataFrame(final_data, columns = ["rule","antecedents", "consequents", "antecedent support", "consequent support","support", "confidence"])  
  return (result)

In [ ]:
#my_asso_rules = create_Assocition_Rules(my_freq_itemset)

In [ ]:
#my_asso_rules.sort_values(by = 'confidence', ascending = True).head(5)

In [ ]:
def recommend(data,antecedent = 3):
  my_freq_itemset = create_Frequent_Itemset(data)
  my_asso_rules = create_Assocition_Rules(my_freq_itemset)
  sample = list(my_asso_rules.sample(n = antecedent, axis=0)["antecedents"])
  print("Input: %s " % sample)
  output = []
  for s in sample: 
    for item in (my_asso_rules[my_asso_rules.antecedents == s]["consequents"].values):
      output.append(item) 
  print("Output: %s" %set(output))
  return 

In [ ]:
start = time.time()
recommend(transaction,antecedent = 3)
print("\nExecute time: ", time.time() - start)

Input: [('Undertale', 'Left 4 Dead 2'), ('Fallout New Vegas Dead Money', 'Company of Heroes (New Steam Version)'), ('Magicka', 'Fallout New Vegas Dead Money')] 
Output: {('Psychonauts',), ('FINAL FANTASY VII',), ("Sid Meier's Civilization V",), ('Undertale',), ('Fallout New Vegas',), ('Serious Sam 3 BFE',), ('Psychonauts Demo',), ('Company of Heroes (New Steam Version)',), ('Skullgirls Endless Beta',), ('Killing Floor Mod Defence Alliance 2',), ('Fallout New Vegas Dead Money',), ('Company of Heroes Opposing Fronts',), ('Company of Heroes',), ("Fallout New Vegas Courier's Stash",), ('Magicka',), ('Left 4 Dead 2',), ('Killing Floor',), ('L.A. Noire',), ('Company of Heroes Tales of Valor',), ('Fallout New Vegas Honest Hearts',), ("Assassin's Creed II",)}

Execute time:  51.16095304489136
